In [1]:
import torch
import torch.nn as nn
import numpy as np
import math

from datasets.openml_datasets import OpenML_Dataset
from datasets.torch_datasets import Torch_Dataset
from utils.weakener import Weakener
from models.model import MLP
from models.general_model import MLP
from utils.losses import PartialLoss,CELoss
from utils.trainig_testing import train_and_evaluate,warm_up,ES_train_and_evaluate

In this case we will create 4 data structures so they contain the 4 Weak structures that support the 4 reconstruction matrices

In [2]:
rec_names = ['_pinv', '_opt', '_conv', '_optconv']
batch_size = 16
dataset_objects = []
weakener_objects = []

# Create dataset objects
for name in rec_names:
    dataset_name = 'Data' + name
    dataset = Torch_Dataset('mnist', batch_size=batch_size)
    dataset_objects.append(dataset)
    globals()[dataset_name] = dataset  # store dataset object in a variable with the corresponding name

# Create weakener objects
for name in rec_names:
    dataset_name = 'Data' + name
    weakener_name = 'Weak' + name
    weakener = Weakener(globals()[dataset_name].num_classes)
    weakener_objects.append(weakener)
    globals()[weakener_name] = weakener

In [3]:
rec_params = {'_pinv':{'optimize':False,'convex':False},
              '_opt':{'optimize':True,'convex':False},
              '_conv':{'optimize':False,'convex':True},
              '_optconv':{'optimize':True,'convex':True}}
for i,name in enumerate(rec_names):
    weakener_objects[i].generate_M('pll')
    _,train_y,_,_ =  globals()['Data'+name].get_data()
    #_,train_y,_,_ =  dataset_objects[i].get_data()
    _,_ = globals()['Weak'+name].generate_weak(train_y)
    #_,_ = weakener_objects[i].generate_weak(train_y)
    globals()['Weak'+name].virtual_matrix(convex=rec_params[name]['convex'],
                                       optimize=rec_params[name]['optimize'])
    #weakener_objects[i].virtual_matrix(convex=rec_params[name]['convex'],
    #                                   optimize=rec_params[name]['optimize'])
    globals()['Weak'+name].virtual_labels()
    #weakener_objects[i].virtual_labels()
    globals()['Data'+name].include_weak(globals()['Weak'+name].v)
    #dataset_objects[i].include_weak(weakener_objects[i].v)

In [4]:
def weight_allocation(net,weights):
    for name, param in net.named_parameters():
        w = name.split('.')
        w_name = 'fc'+str(int(w[1])+1)+'.'+w[2]
        param.data = weights[w_name]
    return net

In [5]:
#We will do the trainig for all the models
all_result = {}
all_model = {}
for i,name in enumerate(rec_names):
    trainloader, testloader = globals()['Data'+name].get_dataloader()
    mlp_feature = MLP(globals()['Data'+name].num_features,
                      [globals()['Data'+name].num_features, globals()['Data'+name].num_features, globals()['Data'+name].num_features],
                      globals()['Data'+name].num_classes, dropout_p = 0.0, bn = False, seed = 1,
                  layer_init = lambda x: nn.init.kaiming_uniform_(x, a=math.sqrt(5))
                      )
    valen_weights = torch.load('results/mnist_random')
    mlp_feature = weight_allocation(mlp_feature,valen_weights)
    optim = torch.optim.Adam(mlp_feature.parameters(),lr=1e-2)
    loss = CELoss()
    print('\nTraining for'+ name +'reconstruction starts:\n')
    mlp, results = ES_train_and_evaluate(mlp_feature,trainloader,testloader,
                                      optimizer=optim,loss_fn=loss,num_epochs=100)
    all_model[name] = {'model':mlp,'optimizer':optim,'loss':loss}
    all_result[name] = results



Training for_pinvreconstruction starts:

Epoch 1/100: Train Loss: 5.6120, Train Acc: 0.1068, Test Acc: 0.1135
Epoch 2/100: Train Loss: 2.3071, Train Acc: 0.1041, Test Acc: 0.1010
Epoch 3/100: Train Loss: 2.3067, Train Acc: 0.1042, Test Acc: 0.1032
Epoch 4/100: Train Loss: 2.3069, Train Acc: 0.1053, Test Acc: 0.1135
Epoch 5/100: Train Loss: 2.3072, Train Acc: 0.1042, Test Acc: 0.0974
Epoch 6/100: Train Loss: 2.3072, Train Acc: 0.1043, Test Acc: 0.1135
Epoch 7/100: Train Loss: 2.3064, Train Acc: 0.1051, Test Acc: 0.1135
Epoch 8/100: Train Loss: 2.3076, Train Acc: 0.1052, Test Acc: 0.1135
Epoch 9/100: Train Loss: 2.3072, Train Acc: 0.1051, Test Acc: 0.1032
Epoch 10/100: Train Loss: 2.3077, Train Acc: 0.1043, Test Acc: 0.1135
Epoch 11/100: Train Loss: 2.3074, Train Acc: 0.1041, Test Acc: 0.1028
Epoch 12/100: Train Loss: 2.3052, Train Acc: 0.1026, Test Acc: 0.0982
Epoch 13/100: Train Loss: 2.3080, Train Acc: 0.1053, Test Acc: 0.1010
Epoch 14/100: Train Loss: 2.3077, Train Acc: 0.1050, Test

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D

tableau_palette = mcolors.TABLEAU_COLORS

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
colors = list(tableau_palette.values())

# Plot the first subplot (Train Loss)
for i,name in enumerate(rec_names):
    ax1.plot(all_result[name]['train_loss'],label='Train'+name,color=colors[i])
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.set_title('Train Loss')


# Plot the second subplot (Accuracies)
for i,name in enumerate(rec_names):
    ax2.plot(all_result[name]['train_acc'], label='Train Accuracy'+name,
             color=colors[i])
    ax2.plot(all_result[name]['test_acc'], label='Test Accuracy'+name,
             color=colors[i],linestyle='--')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Accuracies')
#ax2.legend()
legend_elements = [Line2D([0], [0], linestyle='-', color='black', label='Train Accuracy'),
                   Line2D([0], [0], linestyle='--', color='black', label='Test Accuracy')]
ax2.legend(handles=legend_elements)

fig.suptitle('CE Learning Results', fontsize=18, fontweight='bold', y=1.05)
# Show the plot
plt.show()